# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [7]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'iq-200415-1449'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [8]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

50 training jobs have completed


In [9]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2020, 4, 15, 15, 36, 56, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'test:RMSE',
                                                 'Value': 2.1522631645202637},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2020, 4, 15, 15, 56, 33, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:eu-central-1:964501460451:training-job/iq-200415-1449-040-c7bca260',
 'TrainingJobName': 'iq-200415-1449-040-c7bca260',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2020, 4, 15, 15, 39, 6, tzinfo=tzlocal()),
 'TunedHyperParameters': {'context_length': '26',
                          'dropout_rate': '0.2438733013393834',
                          'learning_rate': '0.023849338036851114',
                          'mini_batch_size': '128',
                          'num_cells': '76'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [10]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 50
{'lowest': 2.1522631645202637, 'highest': 143.34884643554688}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,context_length,dropout_rate,learning_rate,mini_batch_size,num_cells
10,2.152263,1047.0,2020-04-15 15:56:33+00:00,iq-200415-1449-040-c7bca260,Completed,2020-04-15 15:39:06+00:00,26.0,0.243873,0.023849,128.0,76.0
33,2.405342,484.0,2020-04-15 15:16:46+00:00,iq-200415-1449-017-9f8693c1,Completed,2020-04-15 15:08:42+00:00,36.0,0.042917,0.017317,36.0,123.0
28,2.815649,726.0,2020-04-15 15:29:44+00:00,iq-200415-1449-022-8da1ccb2,Completed,2020-04-15 15:17:38+00:00,27.0,0.108373,0.007647,97.0,89.0
1,3.056793,1334.0,2020-04-15 16:13:34+00:00,iq-200415-1449-049-b2c7d34e,Completed,2020-04-15 15:51:20+00:00,39.0,0.031612,0.002433,127.0,103.0
25,3.356047,567.0,2020-04-15 15:28:33+00:00,iq-200415-1449-025-3489f47d,Completed,2020-04-15 15:19:06+00:00,36.0,0.030000,0.001842,34.0,128.0
35,4.423976,745.0,2020-04-15 15:16:02+00:00,iq-200415-1449-015-4c30ea55,Completed,2020-04-15 15:03:37+00:00,27.0,0.085047,0.004989,80.0,79.0
15,4.569050,406.0,2020-04-15 15:37:30+00:00,iq-200415-1449-035-643be656,Completed,2020-04-15 15:30:44+00:00,156.0,0.051791,0.028824,37.0,37.0
8,4.860971,700.0,2020-04-15 15:51:45+00:00,iq-200415-1449-042-d65b52b1,Completed,2020-04-15 15:40:05+00:00,89.0,0.067760,0.001357,37.0,119.0
31,5.096531,783.0,2020-04-15 15:25:49+00:00,iq-200415-1449-019-3b1006d3,Completed,2020-04-15 15:12:46+00:00,27.0,0.090447,0.004989,82.0,79.0
42,5.333425,1215.0,2020-04-15 15:12:18+00:00,iq-200415-1449-008-923efd65,Completed,2020-04-15 14:52:03+00:00,130.0,0.064881,0.001766,78.0,94.0


In [14]:
df  = df.sort_values('FinalObjectiveValue')[:-2]

## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [15]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [16]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))